In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import element_to_be_clickable, presence_of_element_located
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv 
import pandas as pd
import time
import math

In [2]:
# options
options = Options()
options.add_argument('--headless')
options.add_argument("--window-size=1000,1000")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')   

# Populations of Towns

<font size="4"> The goal is to find the total population of every town in *final_data_CD.csv.* </font> <br>
* **Input**
    * ```final_data_CD.csv``` <br>
       - This contains the continental debt held by individuals. For this particular project, we only care about the towns that these individuals are from. </a><br>
* **Output**
    * ```town_pops_clean.csv``` <br>
       - This file will contain the populations of every town. 

In [35]:
cd_df = pd.read_csv("final_data_CD.csv")
towns = cd_df[['Group State', 'Group County', 'Group Town']].drop_duplicates().dropna().reset_index()
towns.drop(columns={"index"}, inplace=True) 
print(towns)

    Group State           Group County       Group Town
0            RI         Bristol County          Bristol
1            CT        Hartford County         Hartford
2            RI      Providence County   North Scituate
3            CT       New Haven County        New Haven
4            NH      Rockingham County       Portsmouth
..          ...                    ...              ...
460          PA  Northumberland County          Sunbury
461          PA     York County County             Fawn
462          PA  Chester County County       New London
463          NY   Queens County County  South Hempstead
464          VA     King George County      King George

[465 rows x 3 columns]


In [36]:
# drop states where census data ancestry does not have records for: VA, GA, NJ, DE
towns_c = towns.copy()
towns = towns.drop(towns[towns['Group State'].isin(['VA', 'GA', 'NJ', 'DE'])].index)
towns_d = towns_c[~towns_c.index.isin(towns.index)]
print(towns_d)

    Group State         Group County      Group Town
82           VA         Grant County      Petersburg
186          NJ      Cape May County        Cape May
188          GA       Chatham County        Savannah
222          VA      Accomack County         Accomac
227          VA  Chesterfield County        Richmond
230          VA  Spotsylvania County  Fredericksburg
241          NJ     Middlesex County       Princeton
261          GA      Richmond County         Augusta
267          NJ    Burlington County      Burlington
291          NJ     Middlesex County       Middlesex
296          VA       Norfolk County      Portsmouth
297          NJ     Hunterdon County      Huntington
299          NJ     Middlesex County   New Brunswick
398          NJ    Burlington County     Mount Holly
401          NJ     Hunterdon County         Trenton
409          VA       Fairfax County      Alexandria
420          NJ         Essex County          Newark
424          VA    James City County    Willia

In [3]:
# dictionary of state codes to actual state names (avoids potential errors)
codes = {
    'CT':'Connecticut',
    'ME':'Maine',
    'MD':'Maryland',
    'MA':'Massachusetts',
    'NH':'New Hampshire',
    'NY':'New York',
    'NC':'North Carolina',
    'PA':'Pennsylvania',
    'RI':'Rhode Island',
    'SC':'South Carolina',
    'VT':'Vermont',
    'VA':'Virginia',
    'GA':'Georgia',
    'NJ':'New Jersey',
    'DE':'Deleware'
}

In [4]:
# handle naming exceptions
exceptions = {
    'Philadelphia County':'Philadelphia',
    'Charleston County':'Charleston',
    'New Haven County':'New Haven'
}

## Accessing Ancestry

1. Access the 1790 census on Ancestry.com using the Selenium library. 
2. Handle county names. Some county names have '[county name] County County'.
3. Access searchbar using Selenium. Selenium inputs [town, county, state, USA] into searchbar. 
4. Selenium clicks on the "Search" button and waits for 0.75 seconds. 
5. Once new webpage is loaded, ```&event_x=0-0-0_1-0```, is added to the current url. This restricts the search only to the town and excludes surrounding counties/towns. 
6. Selenium finds the total number of results, which corresponds to the population of the town. This population is added to the ```town_pops``` dictionary. If there are no results for that town, 'NR' is added to the dictionary instead. 
7. Steps 1-6 are repeated for every town. 

<span style="color: red;">**Note: Runtime will be long. There were approximately ~400 unique towns.**</span>

In [5]:
def ancestry(error_handling, towns_l):
    town_pops = {} # dictionary of all the populations of each town 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 30)
    census_url = "https://www.ancestry.com/search/collections/5058/"
    for town in towns_l:
        # handle '[name] county county'

        if not error_handling:
            county = town[2][:-7].replace(' County', '')
            loc_add = town[3] + ", " + county + ", " + codes[town[1]] + ", USA "
        else:
            county = town[2]
            loc_add = town[3] + ", " + county + ", " + town[1] + ", USA "

        try:
            # open 1790 census 
            try:
                driver.get(census_url)
            except:
                driver.close()
                driver.get(census_url)   

            # handle some exceptions to county names 
            if county in exceptions:
                county = exceptions[county]

            print("-------------------------")
            print(loc_add) ## 

            # wait until event searchbar is visible, then click on it: handles error i noticed
            xpath = "/html/body/div[3]/div/div/div/div/section/div/div/div/div/div/form/div[1]/div/fieldset[2]/div[2]/div/input"
            try:
                wait.until(element_to_be_clickable((By.XPATH, xpath))).click()
            except: # mostly to handle timeout exceptions: close tab and try again
                driver.close()
                driver.get(census_url)
                time.sleep(0.25)
                wait.until(element_to_be_clickable((By.XPATH, xpath))).click()

            input_t = driver.find_element(By.XPATH, xpath)
            input_t.send_keys(loc_add)
            time.sleep(0.75)

            # click on search button 
            wait.until(element_to_be_clickable((By.XPATH, "/html/body/div[3]/div/div/div/div/section/div/div/div/div/div/form/div[1]/div/div[9]/div[1]/input"))).click()
            time.sleep(0.75)

            # add restrictions: we want exact town population
            print(driver.current_url) ##
            driver.get(driver.current_url + "&event_x=0-0-0_1-0")

            # check if town is correct: handles error i noticed
            title_pl = wait.until(presence_of_element_located((By.XPATH, "//*[@id='refineView']/form/div[3]/div/div[1]/div"))).get_attribute("title")
            if (title_pl + " " == loc_add):
                # add to town_pops 
                try:
                    pop = driver.find_element(By.CLASS_NAME, "resultsLabel")
                    town_pops[loc_add] = int(pop.text.split("of")[1].replace(',','')) 
                except NoSuchElementException:
                    town_pops[loc_add] = "NR"
            else:

                print("Titles don't match")
                print("title on page = " + title_pl)
                town_pops[loc_add] = "NR"

            print(town_pops[loc_add]) ##
            print(len(town_pops)) ##
            print(driver.current_url) ##
            print("-------------------------")
            #driver.close() # close current tab 
            #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # restart chrome 
            #wait = WebDriverWait(driver, 30)
        except Exception as e:
            print("___________________________________")
            print("ERROR! Moving swiftly to next town")
            print(e)
            print("___________________________________")
            town_pops[loc_add] = 'ER'
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # restart chrome 
            wait = WebDriverWait(driver, 30)
            continue
    
    return town_pops

In [ ]:
town_pops = ancestry(False, towns.values.tolist())

In [218]:
# save town_pops dictionary as a csv file 
df = pd.DataFrame.from_dict(town_pops, orient="index")
df.to_csv("town_pops.csv")
df = pd.read_csv("town_pops.csv", index_col=0)
print(df)
# printing result

                city          county          state country population
0            Bristol         Bristol   Rhode Island    USA         253
1           Hartford        Hartford    Connecticut    USA         666
2     North Scituate      Providence   Rhode Island    USA          NR
3          New Haven       New Haven    Connecticut    USA         919
4         Portsmouth      Rockingham  New Hampshire    USA         893
..               ...             ...            ...     ...        ...
398          Stephen     Westchester       New York    USA          NR
399          Sunbury  Northumberland   Pennsylvania    USA          NR
400             Fawn            York   Pennsylvania    USA         230
401       New London         Chester   Pennsylvania    USA         120
402  South Hempstead          Queens       New York    USA          NR

[403 rows x 5 columns]
Frequency of ER is : 0


## Fix Formatting

Why? As of right now, there are only two columns. The first column has the location, which includes the town, county, state, and country all in one cell. This is not easy to read. Therefore, town, county, state, and country must become their own columns. The next column has the population. There are no column titles either. The code below fixes these issues. 

In [27]:
# read csv 
towns_df = pd.read_csv("town_pops_2.csv")

# split location name into multiple columns 
towns_df = towns_df.assign(**towns_df['Unnamed: 0'].str.split(', ', expand=True).add_prefix('Info_'))

# rename columns
towns_df.rename(columns={'0':'population', 'Info_0':'city', 'Info_1':'county', 
                         'Info_2':'state', 'Info_3':'country'}, inplace=True)

# remove last three columns and original location column
towns_df.drop(columns={"Unnamed: 0"}, inplace=True)

# reorder columns
towns_df = towns_df[['city', 'county', 'state', 'country', 'population']]
print(towns_df)

towns_df.to_csv("town_pops_clean_2.csv")

                           city     county        state country population
0                    Brookfield  Fairfield  Connecticut    USA         189
1                       Danbury  Fairfield  Connecticut    USA         563
2                     Fairfield  Fairfield  Connecticut    USA         707
3                     Greenwich  Fairfield  Connecticut    USA         321
4                    Huntington  Fairfield  Connecticut    USA         476
...                         ...        ...          ...     ...        ...
1316              Weathersfield    Windsor      Vermont    USA         207
1317  Weathersfield;Springfield    Windsor      Vermont    USA         411
1318                    Windham    Windsor      Vermont    USA          NR
1319                    Windsor    Windsor      Vermont    USA         240
1320                  Woodstock    Windsor      Vermont    USA         268

[1321 rows x 5 columns]


## Handle Mistakes

Why? The code isn't perfect. If Selenium crashed, the program marked that town as 'ER' in ```town_pops``` and moved onto the next town. It's time to go back and fix these. Also, there are individual discrepancies that must be taken care of. The code below essentially finds the rows in ```town_pops_clean.csv``` that have either 'ER' or are in the ```er_indexes``` list, which contains individual town indexes. 

In [38]:
# handle individual cases (pass in their indexes)
er_indexes = [901, 926, 992, 1065, 1142]

In [39]:
# find the populations of towns when webdriver crashed
csv_town = pd.read_csv("town_pops_clean_2.csv")
csv_town = csv_town[["Unnamed: 0", "state", "county", "city", "country", "population"]]

# handle errors (rows with 'ER')
errors = csv_town.loc[csv_town['population'].isin(["ER"])].drop(columns={'population', 'country'}).values.tolist()

# handle individual discrepancies 
indiv_disc = csv_town.loc[csv_town.index.isin(er_indexes)].drop(columns={'population', 'country'}).values.tolist()

er_towns = errors + indiv_disc
print(er_towns)

[[901, 'Pennsylvania', 'Chester', 'East Town'], [926, 'Pennsylvania', 'Chester', 'West Town'], [992, 'Pennsylvania', 'Lancaster', 'Salisbury'], [1065, 'Pennsylvania', 'York', 'Fawn'], [1142, 'Vermont', 'Addison', 'Shoreham']]


## Rerunning the Program

At this point, the ```ancestry``` function is ran again: ```error_handling``` is set to ```True``` and ```towns_l``` is now set to ```er_towns```. Once that's done, the towns are replaced with their new populations. 

In [40]:
town_pops = ancestry(True, er_towns)

-------------------------
East Town, Chester, Pennsylvania, USA 
https://www.ancestry.com/search/collections/5058/?event=_East+Town-Chester-Pennsylvania-USA
4464
1
https://www.ancestry.com/search/collections/5058/?event=_East+Town-Chester-Pennsylvania-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
West Town, Chester, Pennsylvania, USA 
https://www.ancestry.com/search/collections/5058/?event=_West+Town-Chester-Pennsylvania-USA
4464
2
https://www.ancestry.com/search/collections/5058/?event=_West+Town-Chester-Pennsylvania-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Salisbury, Lancaster, Pennsylvania, USA 
https://www.ancestry.com/search/collections/5058/?event=_salisbury-lancaster-pennsylvania-usa_14419
207
3
https://www.ancestry.com/search/collections/5058/?event=_salisbury-lancaster-pennsylvania-usa_14419&event_x=0-0-0_1-0
-------------------------
-------------------------
Fawn, York, Pennsylvania, USA 
https://www.ancestry.com/s

In [41]:
towns_d.replace({"Group State": codes}, inplace=True)
towns_d.rename(columns={"Group State":"state", "Group County":"county", "Group Town":"city"}, inplace=True)
towns_d["country"] = "USA"
towns_d["population"] = "NR"
towns_d = towns_d[["city", "county", "state", "country", "population"]]
print(towns_d)

               city               county       state country population
82       Petersburg         Grant County    Virginia     USA         NR
186        Cape May      Cape May County  New Jersey     USA         NR
188        Savannah       Chatham County     Georgia     USA         NR
222         Accomac      Accomack County    Virginia     USA         NR
227        Richmond  Chesterfield County    Virginia     USA         NR
230  Fredericksburg  Spotsylvania County    Virginia     USA         NR
241       Princeton     Middlesex County  New Jersey     USA         NR
261         Augusta      Richmond County     Georgia     USA         NR
267      Burlington    Burlington County  New Jersey     USA         NR
291       Middlesex     Middlesex County  New Jersey     USA         NR
296      Portsmouth       Norfolk County    Virginia     USA         NR
297      Huntington     Hunterdon County  New Jersey     USA         NR
299   New Brunswick     Middlesex County  New Jersey     USA    

/var/folders/yl/q3pyym191yb9gkpy29l174z9tfwfvv/T/ipykernel_27093/3354630345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  towns_d.rename(columns={"Group State":"state", "Group County":"county", "Group Town":"city"}, inplace=True)
/var/folders/yl/q3pyym191yb9gkpy29l174z9tfwfvv/T/ipykernel_27093/3354630345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  towns_d["country"] = "USA"


In [41]:
# after rerunning program
print(town_pops) # should print out the new populations for each wrong town 
town_copy = pd.read_csv("town_pops_clean_2.csv")
town_copy["location"] = town_copy["city"] + ", " + town_copy["county"] + ", " + town_copy["state"] + ", " + town_copy["country"]
print(town_copy[["population", "location"]]) 

for town in town_pops.keys():
    town_copy.loc[town_copy["location"] == town, "population"] = town_pops[town]
town_copy.drop(columns={"Unnamed: 0", "location"}, inplace=True)

# town_copy = pd.concat([town_copy, towns_d]).reset_index()

print(town_copy) 

town_copy.to_csv("town_pops_clean_2.csv")

{'East Town, Chester, Pennsylvania, USA ': 4464, 'West Town, Chester, Pennsylvania, USA ': 4464, 'Salisbury, Lancaster, Pennsylvania, USA ': 207, 'Fawn, York, Pennsylvania, USA ': 230, 'Shoreham, Addison, Vermont, USA ': 135}
     population                                           location
0           189           Brookfield, Fairfield, Connecticut, USA 
1           563              Danbury, Fairfield, Connecticut, USA 
2           707            Fairfield, Fairfield, Connecticut, USA 
3           321            Greenwich, Fairfield, Connecticut, USA 
4           476           Huntington, Fairfield, Connecticut, USA 
...         ...                                                ...
1316        207              Weathersfield, Windsor, Vermont, USA 
1317        411  Weathersfield;Springfield, Windsor, Vermont, USA 
1318         NR                    Windham, Windsor, Vermont, USA 
1319        240                    Windsor, Windsor, Vermont, USA 
1320        268                  Wood

In [42]:
# find the frequencies of "no records"
town_pop_csv = pd.read_csv("town_pops_clean_2.csv")
res = sum(x == 'NR' for x in town_pop_csv["population"].values.tolist())
print("Frequency of NR is : " + str(res))

Frequency of NR is : 266


# Part II

Get all towns from Ancestry. Run the same program, again. This will prevent us from having to rerun the program again when Assumed State Debt is added. 

In [17]:
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException

In [18]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)
census_url = "https://www.ancestry.com/search/collections/5058/"
driver.get(census_url)

In [19]:
list_obj = driver.find_element(By.XPATH, "//*[@id='browseOptions0']")
select_obj = Select(list_obj)
state_op = select_obj.options
del state_op[0] # remove "Choose..."
states_c = {} # we'll add the counties that are in each state, here 
for state in state_op:
    states_c[state.text] = {}
print(states_c)

{'Connecticut': {}, 'Maine': {}, 'Maryland': {}, 'Massachusetts': {}, 'New Hampshire': {}, 'New York': {}, 'North Carolina': {}, 'Pennsylvania': {}, 'Rhode Island': {}, 'South Carolina': {}, 'Vermont': {}}


In [20]:
# go through each state (X)
# use selenium to find all the options for counties (X)
# store counties in the 'counties' dictionary (X) 

for state in states_c.keys():
    try:
        select_obj.select_by_visible_text(state)    
    except:
        list_obj = driver.find_element(By.XPATH, "//*[@id='browseOptions0']")
        select_obj = Select(list_obj)
        select_obj.select_by_visible_text(state)    
            
    time.sleep(0.25)
    county_ele = wait.until(presence_of_element_located((By.XPATH, "//*[@id='browseOptions1']")))
    county_obj = Select(county_ele)
    county_op = county_obj.options
    
    for county in county_op:
        if county.text == "Choose...":
            continue
        else:
            states_c[state][county.text] = []
    
print(states_c)

{'Connecticut': {'Fairfield': [], 'Hartford': [], 'Litchfield': [], 'Middlesex': [], 'New Haven': [], 'New London': [], 'Tolland': [], 'Windham': []}, 'Maine': {'Cumberland': [], 'Hancock': [], 'Lincoln': [], 'Washington': [], 'York': []}, 'Maryland': {'Anne Arundel': [], 'Baltimore': [], 'Cecil': [], 'Charles': [], 'Dorchester': [], 'Frederick': [], 'Harford': [], 'Kent': [], 'Montgomery': [], 'Prince Georges': [], 'Queen Annes': [], 'St Marys': [], 'Talbot': [], 'Washington': [], 'Worcester': []}, 'Massachusetts': {'Barnstable': [], 'Berkshire': [], 'Bristol': [], 'Dukes': [], 'Essex': [], 'Hampshire': [], 'Middlesex': [], 'Nantucket': [], 'Plymouth': [], 'Suffolk': [], 'Worcester': []}, 'New Hampshire': {'Cheshire': [], 'Grafton': [], 'Hillsborough': [], 'Rockingham': [], 'Strafford': []}, 'New York': {'Albany': [], 'Clinton': [], 'Columbia': [], 'Dutchess': [], 'Kings': [], 'Montgomery': [], 'New York': [], 'Ontario': [], 'Orange': [], 'Queens': [], 'Richmond': [], 'Suffolk': [], '

In [21]:
# go through each county in each state (x)
# use selenium find towns in each county 
# add to dictionary 
# https://github.com/SeleniumHQ/selenium/issues/7186 

for state in states_c.keys():
        state_ele = wait.until(presence_of_element_located((By.XPATH, "//*[@id='browseOptions0']")))
        select_obj = Select(state_ele)
        select_obj.select_by_visible_text(state)  
        time.sleep(0.25)
        
        for county in states_c[state].keys():
            while True:
                maxIterations = 10
                try:
                    county_ele = wait.until(presence_of_element_located((By.XPATH, "//*[@id='browseOptions1']")))
                    select = Select(county_ele)
                    select.select_by_visible_text(county)
                    break
                except (StaleElementReferenceException, NoSuchElementException) as error:
                    maxIterations -= 1
                    if maxIterations == 0:
                        throw
                
            # get list of towns 
            time.sleep(1)
            town_ele = wait.until(presence_of_element_located((By.XPATH, "//*[@id='browselevel2']")))
            town_ele_t = town_ele.text
            print(state + ", " + county)
            print("______________")
            
            if town_ele_t is None:
                print("town_ele is None\n")
                time.sleep(1)
                town_ele = wait.until(presence_of_element_located((By.XPATH, "//*[@id='browselevel2']")))
            
            town_l = town_ele_t.split("\n")
            
            for town in town_l:
                town_x = town.strip()
                
                if town_x != "Not Stated":
                    states_c[state][county].append(town_x)
                    
            print(len(states_c[state][county]))
            print("\n")

print(states_c)

Connecticut, Fairfield
______________
13


Connecticut, Hartford
______________
14


Connecticut, Litchfield
______________
11


Connecticut, Middlesex
______________
6


Connecticut, New Haven
______________
13


Connecticut, New London
______________
0


Connecticut, Tolland
______________
9


Connecticut, Windham
______________
14


Maine, Cumberland
______________
26


Maine, Hancock
______________
25


Maine, Lincoln
______________
51


Maine, Washington
______________
4


Maine, York
______________
28


Maryland, Anne Arundel
______________
0


Maryland, Baltimore
______________
6


Maryland, Cecil
______________
15


Maryland, Charles
______________
0


Maryland, Dorchester
______________
0


Maryland, Frederick
______________
0


Maryland, Harford
______________
0


Maryland, Kent
______________
0


Maryland, Montgomery
______________
0


Maryland, Prince Georges
______________
0


Maryland, Queen Annes
______________
0


Maryland, St Marys
______________
1


Maryland, Talbot
_

In [22]:
# create list of towns
# go through entire dictionary 
# append each town into the list 

towns_an = []

for state in states_c.keys():
    for county in states_c[state].keys():
        for town in states_c[state][county]:
            towns_an.append([0, state, county, town])
            
print(towns_an)

[[0, 'Connecticut', 'Fairfield', 'Brookfield'], [0, 'Connecticut', 'Fairfield', 'Danbury'], [0, 'Connecticut', 'Fairfield', 'Fairfield'], [0, 'Connecticut', 'Fairfield', 'Greenwich'], [0, 'Connecticut', 'Fairfield', 'Huntington'], [0, 'Connecticut', 'Fairfield', 'New Fairfield'], [0, 'Connecticut', 'Fairfield', 'Newtown'], [0, 'Connecticut', 'Fairfield', 'Norwalk and Stamford'], [0, 'Connecticut', 'Fairfield', 'Reading'], [0, 'Connecticut', 'Fairfield', 'Ridgefield'], [0, 'Connecticut', 'Fairfield', 'Stamford'], [0, 'Connecticut', 'Fairfield', 'Stratford'], [0, 'Connecticut', 'Fairfield', 'Weston'], [0, 'Connecticut', 'Hartford', 'Berlin'], [0, 'Connecticut', 'Hartford', 'Bristol'], [0, 'Connecticut', 'Hartford', 'East Hartford'], [0, 'Connecticut', 'Hartford', 'East Windsor'], [0, 'Connecticut', 'Hartford', 'Enfield'], [0, 'Connecticut', 'Hartford', 'Farmington'], [0, 'Connecticut', 'Hartford', 'Glastonbury'], [0, 'Connecticut', 'Hartford', 'Granby'], [0, 'Connecticut', 'Hartford', 'H

In [23]:
town_pops = ancestry(error_handling=True, towns_l=towns_an) 

-------------------------
Brookfield, Fairfield, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_brookfield-fairfield-connecticut-usa_57
189
1
https://www.ancestry.com/search/collections/5058/?event=_brookfield-fairfield-connecticut-usa_57&event_x=0-0-0_1-0
-------------------------
-------------------------
Danbury, Fairfield, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_danbury-fairfield-connecticut-usa_60
563
2
https://www.ancestry.com/search/collections/5058/?event=_danbury-fairfield-connecticut-usa_60&event_x=0-0-0_1-0
-------------------------
-------------------------
Fairfield, Fairfield, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_fairfield-fairfield-connecticut-usa_63
707
3
https://www.ancestry.com/search/collections/5058/?event=_fairfield-fairfield-connecticut-usa_63&event_x=0-0-0_1-0
-------------------------
-------------------------
Greenwich, Fairfield, Connecticut, USA 
https://www.a

-------------------------
Hartford, Hartford, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_hartford-hartford-connecticut-usa_999
666
22
https://www.ancestry.com/search/collections/5058/?event=_hartford-hartford-connecticut-usa_999&event_x=0-0-0_1-0
-------------------------
-------------------------
Simsbury, Hartford, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_simsbury-hartford-connecticut-usa_2336
424
23
https://www.ancestry.com/search/collections/5058/?event=_simsbury-hartford-connecticut-usa_2336&event_x=0-0-0_1-0
-------------------------
-------------------------
Southington, Hartford, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_southington-hartford-connecticut-usa_2439
389
24
https://www.ancestry.com/search/collections/5058/?event=_southington-hartford-connecticut-usa_2439&event_x=0-0-0_1-0
-------------------------
-------------------------
Suffield, Hartford, Connecticut, USA 
https://

209
48
https://www.ancestry.com/search/collections/5058/?event=_Durham-New+Haven-Connecticut-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
East Haven, New Haven, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_East+Haven-New+Haven-Connecticut-USA_3333
169
49
https://www.ancestry.com/search/collections/5058/?event=_east+haven-new+haven-connecticut-usa_3333&event_x=0-0-0_1-0
-------------------------
-------------------------
Guilford, New Haven, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_guilford-new+haven-connecticut-usa_3335
731
50
https://www.ancestry.com/search/collections/5058/?event=_guilford-new+haven-connecticut-usa_3335&event_x=0-0-0_1-0
-------------------------
-------------------------
Hamden, New Haven, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_hamden-new+haven-connecticut-usa_3336
291
51
https://www.ancestry.com/search/collections/5058/?event=_hamden-new+

-------------------------
Canterbury, Windham, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_canterbury-windham-connecticut-usa_3435
288
69
https://www.ancestry.com/search/collections/5058/?event=_canterbury-windham-connecticut-usa_3435&event_x=0-0-0_1-0
-------------------------
-------------------------
Hampton, Windham, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_hampton-windham-connecticut-usa_3446
201
70
https://www.ancestry.com/search/collections/5058/?event=_hampton-windham-connecticut-usa_3446&event_x=0-0-0_1-0
-------------------------
-------------------------
Killingley, Windham, Connecticut, USA 
https://www.ancestry.com/search/collections/5058/?event=_Killingley-Windham-Connecticut-USA
326
71
https://www.ancestry.com/search/collections/5058/?event=_Killingley-Windham-Connecticut-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Lebanon, Windham, Connecticut, USA 
https://www.ancestry.com/se

788
91
https://www.ancestry.com/search/collections/5058/?event=_Gorham+and+Scarborough-Cumberland-Maine-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Gray, Cumberland, Maine, USA 
https://www.ancestry.com/search/collections/5058/?event=_gray-cumberland-maine-usa_3602
89
92
https://www.ancestry.com/search/collections/5058/?event=_gray-cumberland-maine-usa_3602&event_x=0-0-0_1-0
-------------------------
-------------------------
Harpswell, Cumberland, Maine, USA 
https://www.ancestry.com/search/collections/5058/?event=_harpswell-cumberland-maine-usa_3603
177
93
https://www.ancestry.com/search/collections/5058/?event=_harpswell-cumberland-maine-usa_3603&event_x=0-0-0_1-0
-------------------------
-------------------------
Little Falls, Cumberland, Maine, USA 
https://www.ancestry.com/search/collections/5058/?event=_little+falls-cumberland-maine-usa_3605
117
94
https://www.ancestry.com/search/collections/5058/?event=_little+falls-cumberland-maine-usa_3605&event

...

5
1223
https://www.ancestry.com/search/collections/5058/?event=_Greensburrough-Orange-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Groton, Orange, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Groton-Orange-Vermont-USA
8
1224
https://www.ancestry.com/search/collections/5058/?event=_Groton-Orange-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Guildhall, Orange, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Guildhall-Orange-Vermont-USA
27
1225
https://www.ancestry.com/search/collections/5058/?event=_Guildhall-Orange-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Hardwick, Orange, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Hardwick-Orange-Vermont-USA
2
1226
https://www.ancestry.com/search/collections/5058/?event=_Hardwick-Orange-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-----------------------

-------------------------
Benson, Rutland, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_benson-rutland-vermont-usa_5795
130
1250
https://www.ancestry.com/search/collections/5058/?event=_benson-rutland-vermont-usa_5795&event_x=0-0-0_1-0
-------------------------
-------------------------
Brandon, Rutland, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_brandon-rutland-vermont-usa_5797
117
1251
https://www.ancestry.com/search/collections/5058/?event=_brandon-rutland-vermont-usa_5797&event_x=0-0-0_1-0
-------------------------
-------------------------
Castleton, Rutland, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Castleton-Rutland-Vermont-USA
140
1252
https://www.ancestry.com/search/collections/5058/?event=_Castleton-Rutland-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Chittenden, Rutland, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_chittenden-rutla

https://www.ancestry.com/search/collections/5058/?event=_guilford-windham-vermont-usa_5871
411
1278
https://www.ancestry.com/search/collections/5058/?event=_guilford-windham-vermont-usa_5871&event_x=0-0-0_1-0
-------------------------
-------------------------
Hallifax, Windham, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Hallifax-Windham-Vermont-USA
199
1279
https://www.ancestry.com/search/collections/5058/?event=_Hallifax-Windham-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Hinsdale, Windham, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Hinsdale-Windham-Vermont-USA
76
1280
https://www.ancestry.com/search/collections/5058/?event=_Hinsdale-Windham-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Jamaica, Windham, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_jamaica-windham-vermont-usa_5874
48
1281
https://www.ancestry.com/search/collection

-------------------------
Andover, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_andover-windsor-vermont-usa_5901
47
1297
https://www.ancestry.com/search/collections/5058/?event=_andover-windsor-vermont-usa_5901&event_x=0-0-0_1-0
-------------------------
-------------------------
Barnard, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_barnard-windsor-vermont-usa_5904
132
1298
https://www.ancestry.com/search/collections/5058/?event=_barnard-windsor-vermont-usa_5904&event_x=0-0-0_1-0
-------------------------
-------------------------
Barnet, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Barnet-Windsor-Vermont-USA
NR
1299
https://www.ancestry.com/search/collections/5058/?event=_Barnet-Windsor-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Benson, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Benson-Windsor-Vermont-USA
N

-------------------------
Weathersfield;Springfield, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Weathersfield;Springfield-Windsor-Vermont-USA
411
1318
https://www.ancestry.com/search/collections/5058/?event=_Weathersfield;Springfield-Windsor-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Windham, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_Windham-Windsor-Vermont-USA
NR
1319
https://www.ancestry.com/search/collections/5058/?event=_Windham-Windsor-Vermont-USA&event_x=0-0-0_1-0
-------------------------
-------------------------
Windsor, Windsor, Vermont, USA 
https://www.ancestry.com/search/collections/5058/?event=_windsor-windsor-vermont-usa_5942
240
1320
https://www.ancestry.com/search/collections/5058/?event=_windsor-windsor-vermont-usa_5942&event_x=0-0-0_1-0
-------------------------
-------------------------
Woodstock, Windsor, Vermont, USA 
https://www.ancestry.com/search/co

In [24]:
print(town_pops)

{'Brookfield, Fairfield, Connecticut, USA ': 189, 'Danbury, Fairfield, Connecticut, USA ': 563, 'Fairfield, Fairfield, Connecticut, USA ': 707, 'Greenwich, Fairfield, Connecticut, USA ': 321, 'Huntington, Fairfield, Connecticut, USA ': 476, 'New Fairfield, Fairfield, Connecticut, USA ': 277, 'Newtown, Fairfield, Connecticut, USA ': 446, 'Norwalk and Stamford, Fairfield, Connecticut, USA ': 1840, 'Reading, Fairfield, Connecticut, USA ': 264, 'Ridgefield, Fairfield, Connecticut, USA ': 352, 'Stamford, Fairfield, Connecticut, USA ': 1840, 'Stratford, Fairfield, Connecticut, USA ': 548, 'Weston, Fairfield, Connecticut, USA ': 437, 'Berlin, Hartford, Connecticut, USA ': 'ER', 'Bristol, Hartford, Connecticut, USA ': 441, 'East Hartford, Hartford, Connecticut, USA ': 473, 'East Windsor, Hartford, Connecticut, USA ': 481, 'Enfield, Hartford, Connecticut, USA ': 318, 'Farmington, Hartford, Connecticut, USA ': 443, 'Glastonbury, Hartford, Connecticut, USA ': 468, 'Granby, Hartford, Connecticut, 

In [26]:
# save town_pops dictionary as a csv file 
df = pd.DataFrame.from_dict(town_pops, orient="index")
df.to_csv("town_pops_2.csv")
df = pd.read_csv("town_pops_2.csv", index_col=0)
print(df)
# printing result

                                                     0
Brookfield, Fairfield, Connecticut, USA            189
Danbury, Fairfield, Connecticut, USA               563
Fairfield, Fairfield, Connecticut, USA             707
Greenwich, Fairfield, Connecticut, USA             321
Huntington, Fairfield, Connecticut, USA            476
...                                                ...
Weathersfield, Windsor, Vermont, USA               207
Weathersfield;Springfield, Windsor, Vermont, USA   411
Windham, Windsor, Vermont, USA                      NR
Windsor, Windsor, Vermont, USA                     240
Woodstock, Windsor, Vermont, USA                   268

[1321 rows x 1 columns]
